# FLUX NIMs Interactive Lab

Welcome to hands-on exploration of NVIDIA NIMs with FLUX image generation models!

### What are NVIDIA NIMs?
NVIDIA NIMs (NVIDIA Inference Microservices) are optimized containers that serve AI models with maximum performance. Instead of downloading large model files locally, you connect to pre-optimized endpoints that handle inference with advanced optimizations like quantization (FP8/FP4) and TensorRT acceleration.


### Requirements:
* **Container**: `nvcr.io/nim/black-forest-labs/flux.1-dev:1.1.0` and `nvcr.io/nim/black-forest-labs/flux.1-kontext-dev:1.0.0` (deployed to Lepton cloud)
* **GPUs**: 1x GPU with minimum 12GB VRAM (A10, A100, or H100 recommended for FLUX.1-dev); 15GB VRAM for FLUX.1-kontext-dev
* **Storage**: ~50-60GB for NIM containers when deployed to cloud endpoints
* **Shared Memory**: N/A (using cloud-deployed endpoints)
* **External Accounts**: 
  - [NGC API Key](https://ngc.nvidia.com/) (required for NIM container access)
  - [Hugging Face Token](https://huggingface.co/settings/tokens) (required for FLUX model authentication)
  - [Lepton AI Account](https://dashboard.lepton.ai/) (required for cloud deployment)
  - **Important**: Accept the model licenses on Hugging Face before deployment:
    - [FLUX.1-dev model](https://huggingface.co/black-forest-labs/FLUX.1-dev) (may be gated)
    - [FLUX.1-Kontext-dev model](https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev) (may be gated)

### What You'll Do:
- 🚀 Deploy FLUX.1-dev NIM to the cloud using Lepton
- 🎨 Generate stunning images from text prompts
- 🔧 Experiment with different parameters (steps, seed, prompts)
- ✂️ Edit images with natural language using FLUX Kontext
- 💡 Learn production-ready AI deployment workflows

### Step 1: Environment Setup

#### Background: Understanding the Python Environment

This step prepares your Python environment with the necessary libraries for interacting with NVIDIA NIMs and processing images. Each library serves a specific purpose:

**Core Libraries Explained:**
- **`ipywidgets`**: Creates interactive UI elements (dropdowns, sliders, buttons) within Jupyter notebooks
- **`requests`**: Handles HTTP communication with NIM API endpoints
- **`pillow` (PIL)**: Python Imaging Library for image processing, conversion, and manipulation
- **`numpy`**: Numerical computing library used for efficient array operations on image data
- **`matplotlib`**: Plotting library for displaying images and creating visualizations

**Why These Libraries?**
- **Interactive Experience**: ipywidgets transforms a static notebook into a dynamic interface
- **API Communication**: requests provides reliable HTTP client functionality for NIM endpoints
- **Image Processing**: PIL and numpy handle image format conversions (base64 ↔ binary ↔ display)
- **Visualization**: matplotlib renders images inline and creates performance charts

**What the Code Does:**
1. **Package Installation**: `!pip install` ensures all required packages are available
2. **Import Statements**: Load all necessary modules into the Python namespace
3. **Validation**: Print confirmation that environment setup completed successfully

In [ ]:
# Install required packages for NVIDIA NIM interaction and image processing
# Install node.js v20 or newer
# Install and setup Node.js using nvm
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.40.3/install.sh | bash
!. "$HOME/.nvm/nvm.sh" && nvm install --lts
!. "$HOME/.nvm/nvm.sh" && nvm use --lts
print("✅ Node.js installation complete")

!pip install --upgrade pip
!pip install jupyter jupyterlab ipywidgets requests pillow numpy matplotlib pyyaml leptonai

# Fix widget extensions (run if widgets don't display)
!jupyter nbextension install --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

# Import all required libraries for the notebook
import os          # File system operations
import sys         # System-specific parameters and functions
import json        # JSON data handling for API responses
import time        # Timing operations for performance measurement
import base64      # Image encoding/decoding for API transmission
import io          # Input/output operations for image handling
import subprocess  # Process execution for CLI operations
import requests    # HTTP client for NIM API communication
import numpy as np # Numerical operations on image arrays
from datetime import datetime  # Date/time operations

# Jupyter and visualization imports
import ipywidgets as widgets  # Interactive UI components
from IPython.display import display, HTML, clear_output, Image as IPImage

# Image processing imports
from PIL import Image as PILImage  # Image processing and manipulation
import matplotlib.pyplot as plt   # Plotting and image visualization

print("✅ Environment setup complete!")
print("📚 All libraries imported:")
print("  • ipywidgets: Interactive UI components")
print("  • requests: HTTP communication with NIMs")
print("  • PIL: Image processing and conversion")
print("  • matplotlib: Image display and visualization")
print("  • numpy: Numerical operations on images")
print("  • leptonai: Lepton CLI for endpoint deployment")
print("  • subprocess: CLI command execution")
print("\n💡 If widgets don't display, restart kernel and re-run this cell")

### Step 2: NIM Endpoint Configuration

#### Background: Understanding NVIDIA NIMs Architecture

**What are NVIDIA NIMs?**
NVIDIA NIMs (NVIDIA Inference Microservices) are containerized AI model serving solutions that provide:

- **Optimized Inference**: Models are pre-quantized and optimized with TensorRT for maximum performance
- **Standardized APIs**: Consistent REST API interface across all NIM containers
- **Production Ready**: Built-in scaling, monitoring, and deployment capabilities
- **Hardware Acceleration**: Automatic GPU utilization and memory optimization

**NIM vs Traditional Model Hosting:**
| Traditional Approach | NVIDIA NIMs |
|---------------------|-------------|
| Manual model optimization | Pre-optimized with TensorRT |
| Custom API development | Standardized REST API |
| Complex deployment | Docker container deployment |
| Variable performance | Consistent, optimized performance |

**Deployment Options:**

We'll be using **Custom NIM Endpoints** deployed to cloud platforms:
   - Deploy to cloud platforms (Lepton, AWS, GCP, Azure)
   - Production-ready scaling
   - Remote GPU resources
   - Custom domain/URL access

**What the Configuration Code Does:**
- **Text Input Widget**: Allows entry of custom NIM endpoint URLs
- **Password Widget**: Secure input for NGC API keys (required for container access)
- **Display Logic**: Shows interactive UI elements for endpoint configuration

**NGC API Key Purpose:**
The NGC (NVIDIA GPU Cloud) API key is required to:
- Pull NIM containers from NVIDIA's registry
- Authenticate with deployed NIM endpoints
- Access NVIDIA's optimized model containers

### Step 2.1: Lepton CLI Setup & Authentication

#### One-Click Lepton Setup

This cell installs the Lepton CLI and handles authentication, making the notebook completely self-contained.

In [ ]:
# Lepton CLI Installation and Manual Authentication
# Expected time: 2-3 minutes for installation + manual login

# Create widgets for NGC API key (still needed for NIM containers)
ngc_api_key_widget = widgets.Password(
    placeholder='Enter your NGC API key',
    description='NGC API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

setup_output = widgets.Output()

def test_lepton_command(cmd_path):
    """Test if a lepton command works"""
    try:
        if isinstance(cmd_path, list):
            result = subprocess.run(cmd_path + ['--version'], capture_output=True, text=True, timeout=10)
        else:
            result = subprocess.run([cmd_path, '--version'], capture_output=True, text=True, timeout=10)
        
        if result.returncode == 0:
            return True, result.stdout.strip()
        else:
            return False, f"Command failed: {result.stderr}"
    except Exception as e:
        return False, f"Error testing command: {str(e)}"

def install_lepton_cli():
    """Install Lepton CLI with comprehensive testing"""
    
    print("🔍 Checking Python environment and pip...")
    print(f"   🐍 Python executable: {sys.executable}")
    print(f"   📦 Python version: {sys.version}")
    
    # Check pip version
    try:
        pip_result = subprocess.run([sys.executable, '-m', 'pip', '--version'], 
                                  capture_output=True, text=True, timeout=10)
        if pip_result.returncode == 0:
            print(f"   📋 Pip version: {pip_result.stdout.strip()}")
    except Exception as e:
        print(f"   ⚠️  Could not check pip version: {e}")
    
    print("\n🔍 Checking for existing Lepton CLI...")
    
    # Test different command approaches based on official docs
    test_commands = [
        'lep',  # Official CLI command name
        'lepton',  # Alternative name
        f'{os.path.dirname(sys.executable)}/lep',
        f'{os.path.dirname(sys.executable)}/lep.exe',
        f'{os.path.dirname(sys.executable)}/Scripts/lep.exe',
    ]
    
    for cmd in test_commands:
        print(f"   Testing: {cmd}")
        works, msg = test_lepton_command(cmd)
        if works:
            print(f"   ✅ Found working command: {cmd}")
            return True, f"✅ Lepton CLI found: {msg}", cmd
        else:
            print(f"   ❌ {cmd}: {msg}")
    
    # Install leptonai package with force reinstall
    try:
        print("\n📦 Installing leptonai package... (30-60 seconds)")
        print("   🔄 Using --force-reinstall to ensure latest version")
        
        # First, uninstall any existing version
        print("   🗑️  Uninstalling any existing leptonai...")
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', 'leptonai', '-y'], 
                      capture_output=True, text=True, timeout=30)
        
        # Install latest version with force reinstall and no cache
        result = subprocess.run([
            sys.executable, '-m', 'pip', 'install', 
            '--upgrade', '--force-reinstall', '--no-cache-dir', 
            'leptonai'
        ], capture_output=True, text=True, timeout=120)
        
        if result.returncode != 0:
            return False, f"❌ pip install failed: {result.stderr}", None
        
        print("✅ Package installed successfully")
        
        # Verify the installed version
        try:
            import leptonai
            version = getattr(leptonai, '__version__', 'unknown')
            print(f"   📋 Installed leptonai version: {version}")
        except ImportError:
            print("   ⚠️  Could not import leptonai to check version")
        
        # Test commands again after installation
        print("🔍 Testing commands after installation...")
        for cmd in test_commands:
            print(f"   Testing: {cmd}")
            works, msg = test_lepton_command(cmd)
            if works:
                print(f"   ✅ Working command found: {cmd}")
                return True, f"✅ Lepton CLI installed: {msg}", cmd
            else:
                print(f"   ❌ {cmd}: {msg}")
        
        # Try to find lep executable in pip install location
        print("🔍 Searching for lep executable...")
        try:
            # Get pip install location
            pip_result = subprocess.run([sys.executable, '-m', 'pip', 'show', 'leptonai'], 
                                      capture_output=True, text=True)
            if pip_result.returncode == 0:
                for line in pip_result.stdout.split('\n'):
                    if line.startswith('Location:'):
                        location = line.split(':', 1)[1].strip()
                        print(f"   Package location: {location}")
                        
                        # Try to find bin directory and scripts
                        possible_bins = [
                            os.path.join(location, '..', 'bin', 'lep'),
                            os.path.join(location, '..', '..', 'bin', 'lep'),
                            os.path.join(location, '..', 'Scripts', 'lep.exe'),
                            os.path.join(location, '..', '..', 'Scripts', 'lep.exe'),
                            os.path.join(os.path.dirname(sys.executable), 'lep'),
                            os.path.join(os.path.dirname(sys.executable), 'lep.exe'),
                            os.path.join(os.path.dirname(sys.executable), 'Scripts', 'lep.exe'),
                        ]
                        
                        for bin_path in possible_bins:
                            normalized_path = os.path.normpath(bin_path)
                            print(f"   Checking: {normalized_path}")
                            if os.path.exists(normalized_path):
                                works, msg = test_lepton_command(normalized_path)
                                if works:
                                    print(f"   ✅ Found working executable: {normalized_path}")
                                    return True, f"✅ Lepton CLI found: {msg}", normalized_path
                                else:
                                    print(f"   ❌ Executable found but not working: {msg}")
        except Exception as e:
            print(f"   Error searching for executable: {e}")
        
        return False, "❌ Lepton CLI installed but no working command found. You may need to use Lepton web interface instead.", None
        
    except subprocess.TimeoutExpired:
        return False, "❌ Installation timed out", None
    except Exception as e:
        return False, f"❌ Installation error: {str(e)}", None

def manual_lepton_login(lepton_cmd):
    """Guide user through manual Lepton login process"""
    
    print("🔐 Manual Lepton Authentication Required")
    print("=" * 50)
    print("The automated login approach has limitations.")
    print("Please follow these steps for manual authentication:")
    print()
    
    print("STEP 1: Run the login command")
    print("-" * 30)
    print(f"Command to run: {lepton_cmd} login")
    print()
    
    # Create a code cell that user can copy
    login_command = f"{lepton_cmd} login"
    print("📋 Copy and run this command in a new terminal:")
    print(f"   {login_command}")
    print()
    
    print("STEP 2: Follow the interactive prompts")
    print("-" * 40)
    print("1. Press Enter when prompted")
    print("2. A URL will be displayed - copy it")
    print("3. Open the URL in a new browser tab")
    print("4. Create a token in the Lepton dashboard")
    print("5. Copy the workspace:token from the second 'lep login' command output")
    print("6. Paste it back in the terminal and press Enter")
    print()
    
    print("STEP 3: Verify authentication")
    print("-" * 35)
    print("After successful login, run:")
    print(f"   {lepton_cmd} workspace list")
    print()
    print("You should see your workspace(s) listed.")
    print()
    
    print("STEP 4: Return to this notebook")
    print("-" * 40)
    print("Once authenticated, you can proceed with NIM deployment.")
    print("The deployment cells will use your authenticated CLI session.")
    print()
    
    return True, "Manual login instructions provided"

def setup_lepton_cli(button):
    """Complete Lepton CLI setup process with manual authentication"""
    with setup_output:
        setup_output.clear_output(wait=True)
        
        # Get NGC API key
        ngc_key = ngc_api_key_widget.value.strip()
        
        print("🚀 Setting up Lepton CLI environment...")
        print("⏱️  Expected time: 2-3 minutes")
        
        # Validate NGC API key
        if not ngc_key:
            print("❌ Please enter your NGC API key")
            return
            
        print(f"📋 NGC key length: {len(ngc_key)} characters")
        print()
        
        # Step 1: Install Lepton CLI
        install_success, install_message, lepton_cmd = install_lepton_cli()
        print(f"\n{install_message}")
        
        if not install_success:
            print("\n⚠️  CLI command not available")
            print("💡 You can use Lepton web interface for manual deployment:")
            print("   1. Go to https://dashboard.lepton.ai/")
            print("   2. Click 'Create Endpoint'")
            print("   3. Use these settings:")
            print(f"      - Container: nvcr.io/nim/black-forest-labs/flux.1-dev:1.1.0")
            print(f"      - Environment: NGC_API_KEY={ngc_key}")
            print("   4. Copy the endpoint URL for use in this notebook")
            
            # Store NGC key for manual deployment
            os.environ['NGC_API_KEY'] = ngc_key
            print("\n✅ NGC API key configured for manual deployment")
            return
        
        print(f"🔍 Using Lepton command: {lepton_cmd}")
        
        # Step 2: Guide user through manual authentication
        auth_success, auth_message = manual_lepton_login(lepton_cmd)
        print(f"\n{auth_message}")
        
        if auth_success:
            # Store configuration
            os.environ['NGC_API_KEY'] = ngc_key
            os.environ['LEPTON_CMD'] = str(lepton_cmd)  # Store command path for later use
            
            print("\n🎉 Lepton CLI setup complete!")
            print("💡 After completing manual login, you can deploy NIM endpoints")
            print("💡 Available CLI commands after login:")
            print("   • lep deployment create - Create new deployments")
            print("   • lep deployment list - List existing deployments")
            print("   • lep workspace list - List workspaces")
            print()
            print("⚠️  Remember: You need to complete the manual login process")
            print("   in a terminal before proceeding with deployment.")

# Create setup interface
print("🔧 Lepton CLI Setup & Manual Authentication")
print("⏱️  Expected time: 2-3 minutes")
print("📊 Resources needed: Minimal (CLI installation + manual login)")
print("\n🔑 Required Credentials:")

display(ngc_api_key_widget)

setup_button = widgets.Button(
    description='🚀 Setup Lepton CLI',
    button_style='primary',
    icon='wrench',
    layout=widgets.Layout(width='200px', height='40px')
)

setup_button.on_click(setup_lepton_cli)
display(setup_button)
display(setup_output)

print("\n💡 Where to get credentials:")
print("  • NGC API Key: https://ngc.nvidia.com/ → Generate API Key")
print("\n📋 What this does:")
print("  • Installs leptonai package with: pip install --upgrade --force-reinstall --no-cache-dir leptonai")
print("  • Uninstalls old versions first to ensure clean installation")
print("  • Finds working 'lep' command (official CLI)")
print("  • Provides manual login instructions")
print("  • Configures NGC API key for NIM deployment")
print("\n⚠️  Note: Manual login required - automated authentication has limitations")
print("\n🔄 Installation Process:")
print("  • Uninstalls any existing leptonai package")
print("  • Installs latest version with force reinstall")
print("  • Skips pip cache to ensure fresh download")
print("  • Verifies installation and version")

### Step 2.2: Deploy FLUX NIM Endpoints

#### Automated Endpoint Deployment

Deploy FLUX NIMs as Lepton endpoints directly from the notebook. This creates production-ready API endpoints you can use immediately.

In [ ]:
# FLUX NIM Endpoint Deployment using Lepton CLI
# Expected time: 3-5 minutes per model (deployment + readiness check)
# Resource requirements: 1 GPU per endpoint

# Deployment configuration widgets
nim_model_selector = widgets.Dropdown(
    options=[
        ('FLUX.1 Dev - Text-to-Image Generation', 'flux.1-dev')
    ],
    value='flux.1-dev',
    description='NIM Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

endpoint_name_widget = widgets.Text(
    value='',
    placeholder='my-flux-nim (auto-generated if empty)',
    description='Endpoint Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Resource configuration - user input field
resource_type_widget = widgets.Text(
    value='gpu.1xh200',
    placeholder='e.g., gpu.a10, gpu.a100-40gb, gpu.1xh200',
    description='Resource Shape:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Deployment output
deployment_output = widgets.Output()

def try_deployment_create(model, endpoint_name, resource_type, node_group, hf_token, image_pull_secrets):
    """Create deployment using lep deployment create command"""
    
    lepton_cmd = os.environ.get('LEPTON_CMD', 'lep')
    
    nim_containers = {
        'flux.1-dev': 'nvcr.io/nim/black-forest-labs/flux.1-dev:1.1.0',
        'flux.1-kontext-dev': 'nvcr.io/nim/black-forest-labs/flux.1-kontext-dev:1.0.0'
    }

    container_image = nim_containers[model]
    ngc_key = os.environ.get('NGC_API_KEY')

    print(f"🚀 Creating deployment...")
    print(f"   📦 Container: {container_image}")
    print(f"   🏷️  Name: {endpoint_name}")
    print(f"   🔧 Resources: {resource_type}")
    print(f"   🏢 Node Group: {node_group}")

    # Validate HF_TOKEN
    if not hf_token or not hf_token.strip():
        return False, "❌ HF_TOKEN is required for FLUX NIM deployments. Please provide a valid Hugging Face token."

    try:
        # Set environment variables for the subprocess
        env = os.environ.copy()
        if ngc_key:
            env['NGC_API_KEY'] = ngc_key
        if hf_token:
            env['HF_TOKEN'] = hf_token

        # Build deployment create command
        cmd_parts = [
            lepton_cmd,
            'deployment', 'create',
            '--name', endpoint_name,
            '--container-image', container_image,
            '--container-port', '8000',
            '--resource-shape', resource_type,
            '--replicas-static', '1',
            '--env', f'NGC_API_KEY={ngc_key}',
            '--env', f'HF_TOKEN={hf_token}',
            '--public',
            '--node-group', node_group,
            '--image-pull-secrets', image_pull_secrets
        ]

        print(f"   🔄 Running deployment command...")

        result = subprocess.run(cmd_parts, capture_output=True, text=True, env=env, timeout=120)

        if result.returncode == 0:
            print(f"   ✅ Deployment creation successful!")
            
            # Extract deployment URL from output
            deployment_url = f"https://{endpoint_name}.cloud.lepton.ai"
            return True, f"Deployment created successfully. URL: {deployment_url}"
        else:
            error_msg = result.stderr.strip() or result.stdout.strip()
            
            # Provide helpful error messages
            if 'authentication' in error_msg.lower() or 'token' in error_msg.lower():
                return False, f"❌ Authentication issue. Please run 'lep login' first."
            elif 'no available node groups' in error_msg.lower():
                return False, f"❌ No GPU resources available. Check your Lepton workspace quota."
            else:
                return False, f"Deployment failed: {error_msg}"

    except subprocess.TimeoutExpired:
        return False, "Deployment command timed out after 2 minutes"
    except Exception as e:
        return False, f"Deployment error: {str(e)}"

def start_nim_deployment():
    """Start NIM deployment process"""
    model = nim_model_selector.value
    endpoint_name = endpoint_name_widget.value.strip()
    resource_type = resource_type_widget.value
    node_group = node_group_input.value.strip()
    hf_token = hf_token_input.value.strip()
    image_pull_secrets = image_pull_secrets_input.value.strip()

    with deployment_output:
        deployment_output.clear_output(wait=True)
        
        print("🚀 FLUX NIM Deployment")
        print("="*60)
        
        # Validate required inputs
        if not hf_token or not hf_token.strip():
            print("❌ HF_TOKEN is required!")
            print("📋 Get a token from: https://huggingface.co/settings/tokens")
            return
        
        # Generate endpoint name if not provided
        if not endpoint_name:
            endpoint_name = "flux-dev"
        else:
            # Sanitize endpoint name
            endpoint_name = endpoint_name.replace('.', '-').replace('_', '-').lower()

        # Check if CLI is available
        lepton_cmd = os.environ.get('LEPTON_CMD', '')
        if not lepton_cmd:
            print("❌ Please complete the Lepton CLI setup in the previous cell first")
            return

        print(f"\n🎨 Deploying: {model}")
        print("-" * 40)
        
        # Deploy
        create_success, create_result = try_deployment_create(model, endpoint_name, resource_type, node_group, hf_token, image_pull_secrets)

        if create_success:
            print(f"\n✅ {create_result}")
            print("\n📊 Next Steps:")
            print("  1. Wait 3-5 minutes for deployment to become ready")
            print("  2. Copy the endpoint URL")
            print("  3. Use it in the image generation section below")
        else:
            print(f"\n❌ {create_result}")
            print("\n💡 Troubleshooting:")
            print("  • Ensure you've run 'lep login' in a terminal")
            print("  • Verify your Lepton workspace has available GPU quota")
            print("  • Check that your HF_TOKEN is valid")

# Create deployment interface
print("🚀 FLUX NIM Endpoint Deployment")
print("⏱️  Expected time: 3-5 minutes")
print()
print("📋 Required Configuration:")

display(nim_model_selector)
display(endpoint_name_widget)
display(resource_type_widget)

# Additional required inputs
node_group_input = widgets.Text(
    value='tme-nebius-h200-01',
    placeholder='Enter your node group name',
    description='Node Group:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

hf_token_input = widgets.Password(
    value='',
    placeholder='Enter your Hugging Face token (hf_...)',
    description='HF Token:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

image_pull_secrets_input = widgets.Text(
    value='roclark-ngc',
    placeholder='Enter your image pull secrets name',
    description='Image Pull Secrets:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(node_group_input)
display(hf_token_input)
display(image_pull_secrets_input)

deploy_nim_button = widgets.Button(
    description='🚀 Deploy',
    button_style='success',
    icon='rocket',
    layout=widgets.Layout(width='150px', height='40px')
)

deploy_nim_button.on_click(lambda b: start_nim_deployment())
display(deploy_nim_button)
display(deployment_output)

print("\n💡 Deployment uses lep deployment create")
print("⚠️  Remember: Run 'lep login' in a terminal first!")
print("⚠️  IMPORTANT: Accept FLUX.1-dev model license on Hugging Face first:")
print("   https://huggingface.co/black-forest-labs/FLUX.1-dev")


### Step 3: Generate Images

#### Background: Understanding NIM API Communication

**NIM API Architecture:**
NVIDIA NIMs expose a standardized REST API that follows OpenAI-compatible patterns:

- **Endpoint Pattern**: `/v1/infer` - Standardized inference endpoint
- **HTTP Method**: POST - Sends generation parameters
- **Content Type**: `application/json` - Structured data format
- **Response Format**: JSON with base64-encoded images in `artifacts` array

**API Payload Structure:**
```json
{
  "prompt": "Your text description",
  "mode": "base",           // "base" for generation, "kontext" for editing
  "seed": 12345,           // Random seed for reproducibility
  "steps": 50              // Number of inference iterations
```

**Response Structure:**
```json
{
  "artifacts": [
    {
      "base64": "iVBORw0KGgoAAAANSUhEUgAA...",  // Base64 image data
      "seed": 12345,                            // Actual seed used
      "finish_reason": "SUCCESS"
    }
  ]
```

**Image Processing Pipeline:**
1. **API Request**: Send prompt + parameters to NIM endpoint
2. **Model Inference**: NIM processes request on GPU
3. **Base64 Encoding**: NIM encodes result image as base64 string
4. **Response Parsing**: Extract base64 data from artifacts array
5. **Image Decoding**: Convert base64 back to PIL Image object
6. **Display**: Render image using matplotlib
7. **Storage**: Save image to local filesystem

**Performance Measurement:**
- **Timing**: Measure end-to-end request duration
**Error Handling:**
- **Network Issues**: Timeout, connection errors
- **API Errors**: Invalid parameters, server errors  
- **Image Processing**: Decoding failures, display issues
- **Graceful Degradation**: Continue with other models if one fails

**What the Generation Code Does:**
- **API Communication**: Handles HTTP requests to NIM endpoints
- **Image Processing**: Converts between base64, binary, and display formats  
- **Performance Tracking**: Times each generation for comparison
- **Error Management**: Provides detailed error messages for troubleshooting
- **File Management**: Automatically saves images with metadata
- **UI Integration**: Displays results inline with interactive elements

In [ ]:
# Image generation and comparison function
def generate_with_nim(endpoint, ngc_api_key, model, prompt, steps, seed):
    """Generate image using actual NIM endpoint with correct API format"""
    
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    # Note: This deployment is public, no authentication required
    # Authentication headers removed for public Lepton deployments
    
    # Official NIM API payload format (from NVIDIA documentation)
    payload = {
        "prompt": prompt,
        "mode": "base",  # NIM uses mode parameter
        "seed": seed,
        "steps": steps
    }
    
    start_time = time.time()
    
    try:
        # Use the official NIM API endpoint pattern
        nim_endpoint = endpoint.rstrip('/') + '/v1/infer'
        # Debug output to show what's being sent
        print(f"   🔍 Endpoint: {nim_endpoint}")
        print(f"   📋 Payload: {payload}")
        print(f"   🌐 Public deployment - no authentication required")
        
        response = requests.post(nim_endpoint, headers=headers, json=payload, timeout=120)
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            generation_time = end_time - start_time
            
            # Correct NIM response parsing - images are in artifacts array
            if 'artifacts' in result and len(result['artifacts']) > 0:
                base64_image = result['artifacts'][0].get('base64')
                if base64_image:
                    return {
                        'success': True,
                        'image_base64': base64_image,
                        'generation_time': generation_time,
                        'model': model,
                        'endpoint': endpoint,
                        'seed': result['artifacts'][0].get('seed', seed)
                    }
            
            return {
                'success': False,
                'error': f"No image artifacts in response: {result}",
                'model': model,
                'endpoint': endpoint
            }
        else:
            return {
                'success': False,
                'error': f"NIM API Error: {response.status_code} - {response.text}",
                'model': model,
                'endpoint': endpoint
            }
            
    except Exception as e:
        return {
            'success': False,
            'error': f"Request failed: {str(e)}",
            'model': model,
            'endpoint': endpoint
        }

def display_image_from_base64(base64_data, title="Generated Image"):
    """Convert base64 to PIL Image and display in Jupyter"""
    try:
        # Decode base64 to bytes
        image_bytes = base64.b64decode(base64_data)
        
        # Create PIL Image from bytes
        image = PILImage.open(io.BytesIO(image_bytes))
        
        print(f"🖼️  Displaying image: {title}")
        print(f"   📏 Size: {image.size[0]}x{image.size[1]} pixels")
        
        # Try multiple display methods for better Jupyter compatibility
        try:
            # Method 1: Use matplotlib with proper inline display (most reliable)
            import matplotlib
            matplotlib.use('inline')  # Ensure inline backend
            
            plt.figure(figsize=(10, 10))
            plt.imshow(image)
            plt.axis('off')
            plt.title(title, fontsize=16, fontweight='bold', pad=20)
            plt.tight_layout()
            
            # Force display in current output area
            from IPython.display import display as ipython_display
            ipython_display(plt.gcf())
            plt.close()  # Close to prevent duplicate display
            
            print("   ✅ Image displayed using matplotlib")
        except Exception as e1:
            print(f"   ⚠️  Matplotlib failed: {e1}")
            try:
                # Method 2: Use IPython.display as fallback
                from IPython.display import Image as IPImage, display as ipython_display
                ipython_display(IPImage(data=image_bytes))
                print("   ✅ Image displayed using IPython.display")
            except Exception as e2:
                print(f"   ❌ Both display methods failed: {e2}")
                print("   💡 Image object created successfully (display methods failed)")
        
        return image
    except Exception as e:
        print(f"❌ Error processing image: {e}")
        return None

def save_image_from_base64(base64_data, filepath):
    """Save base64 image data to file"""
    try:
        # Decode base64 to bytes
        image_bytes = base64.b64decode(base64_data)
        
        # Create PIL Image and save
        image = PILImage.open(io.BytesIO(image_bytes))
        image.save(filepath)
        return True
    except Exception as e:
        print(f"❌ Error saving image to {filepath}: {e}")
        return False

def image_to_base64(image_bytes):
    """Convert image bytes to base64 string for API transmission"""
    try:
        # Convert bytes to base64 string
        base64_string = base64.b64encode(image_bytes).decode('utf-8')
        return base64_string
    except Exception as e:
        print(f"Error converting image to base64: {e}")
        return None

# Configuration widgets for image generation
print("🔧 NIM Endpoint Configuration")
print("Configure your NIM endpoints for image generation")
print()

# Endpoint selection dropdown
endpoint_widget = widgets.Dropdown(
    options=[
        ('Custom NIM Endpoint', 'custom')
    ],
    value='custom',
    description='NIM Endpoint:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Custom endpoint input (shown when 'custom' is selected)
custom_endpoint_widget = widgets.Text(
    value='',
    placeholder='Enter your NIM endpoint URL (e.g., https://your-deployment.cloud.lepton.ai)',
    description='Custom URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# NGC API Key input
ngc_api_key_widget = widgets.Password(
    value='',
    placeholder='Enter your NGC API key (optional for public deployments)',
    description='NGC API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(endpoint_widget)
display(custom_endpoint_widget)
display(ngc_api_key_widget)

print()
print("🎨 Image Generation Parameters")

# Test image display function
def test_image_display():
    """Test if image display is working in Jupyter"""
    try:
        # Create a simple test image
        import numpy as np
        from PIL import Image as PILImage
        
        # Create a 100x100 test image with gradient
        test_array = np.zeros((100, 100, 3), dtype=np.uint8)
        test_array[:, :, 0] = np.linspace(0, 255, 100).reshape(1, -1)  # Red gradient
        test_array[:, :, 1] = np.linspace(0, 255, 100).reshape(-1, 1)  # Green gradient
        test_array[:, :, 2] = 128  # Blue constant
        
        test_image = PILImage.fromarray(test_array)
        
        # Convert to base64 for testing
        import io, base64
        buffer = io.BytesIO()
        test_image.save(buffer, format='PNG')
        test_base64 = base64.b64encode(buffer.getvalue()).decode()
        
        print("🧪 Testing image display...")
        display_image_from_base64(test_base64, "Test Image - Gradient Pattern")
        
    except Exception as e:
        print(f"❌ Image display test failed: {e}")

# Uncomment the line below to test image display
# test_image_display()

# Model selection for comparison
# Model selection - FLUX.1-dev only at this stage
model_selector = widgets.Dropdown(
    options=[('FLUX.1 Dev', 'flux.1-dev')],
    value='flux.1-dev',
    description='Model:',
    layout=widgets.Layout(width='300px')
)

# Textarea for detailed prompt input
prompt_widget = widgets.Textarea(
    value='A serene mountain landscape at sunset with dramatic clouds',
    placeholder='Enter your image generation prompt (be detailed for best results)',
    description='Prompt:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='80px')
)

# Slider for controlling inference steps (quality vs speed tradeoff)
steps_widget = widgets.IntSlider(
    value=50,           # Balanced default
    min=1,              # Minimum (very fast, lower quality)
    max=100,            # Maximum (slower, higher quality)
    step=1,
    description='Steps:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Seed input for reproducible results
seed_widget = widgets.IntText(
    value=42,           # Default seed for consistency
    description='Seed:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

display(model_selector)
display(prompt_widget)
display(steps_widget)
display(seed_widget)

def run_generation():
    """Run image generation with NIM endpoint"""
    
    endpoint = endpoint_widget.value
    if endpoint == 'custom':
        endpoint = custom_endpoint_widget.value
        
    ngc_api_key = ngc_api_key_widget.value
    prompt = prompt_widget.value
    steps = steps_widget.value
    seed = seed_widget.value
    model = model_selector.value
    
    if not endpoint.strip():
        print("❌ Please select or enter a NIM endpoint")
        return
        
    if not prompt.strip():
        print("❌ Please enter a prompt")
        return
    
    print(f"🚀 Generating image with {model}...")
    print(f"Endpoint: {endpoint}")
    print(f"Prompt: {prompt}")
    print(f"Parameters: {steps} steps, seed {seed}")
    print("\\n" + "="*50)
    
    print(f"\\n🎨 Generating with {model} via NIM...")
    
    result = generate_with_nim(endpoint, ngc_api_key, model, prompt, steps, seed)
    
    if result['success']:
        print(f"✅ Generated in {result['generation_time']:.2f}s")
        
        # Display the generated image
        image_title = f"{model} - {prompt[:50]}..."
        pil_image = display_image_from_base64(result['image_base64'], image_title)
        
        if pil_image:
            # Save image to outputs directory
            os.makedirs('examples/outputs', exist_ok=True)
            timestamp = int(time.time())
            filename = f"flux_{model}_{timestamp}.png"
            filepath = f"examples/outputs/{filename}"
            
            if save_image_from_base64(result['image_base64'], filepath):
                print(f"\\n💾 Image saved: {filepath}")
            
            print(f"\\n🖼️  Image generated successfully!")
            print("\\n💡 Check examples/outputs/ directory for your image")
    else:
        print(f"❌ Generation failed: {result['error']}")
        print("\\n💡 Troubleshooting:")
        print("  • Check NIM endpoint is correct and accessible")
        print("  • Verify the endpoint is ready (may take 3-5 min after deployment)")
        print("  • Ensure NGC API key is correct if required")


# Create generation button
generate_button = widgets.Button(
    description='🎨 Generate',
    button_style='success',
    icon='play',
    layout=widgets.Layout(width='200px', height='40px')
)

generate_button.on_click(lambda b: run_generation())
display(generate_button)

In [ ]:
# Image gallery viewer for generated results
def display_image_gallery():
    """Display a gallery of previously generated images"""
    
    outputs_dir = 'examples/outputs'
    if not os.path.exists(outputs_dir):
        print("❌ No outputs directory found. Generate some images first!")
        return
    
    # Find all PNG files in outputs directory
    image_files = [f for f in os.listdir(outputs_dir) if f.endswith('.png')]
    
    if not image_files:
        print("❌ No images found in outputs directory. Generate some images first!")
        return
    
    print(f"🖼️  Found {len(image_files)} generated image(s):")
    print("=" * 50)
    
    # Sort by modification time (newest first)
    image_files.sort(key=lambda x: os.path.getmtime(os.path.join(outputs_dir, x)), reverse=True)
    
    # Display images in a grid
    cols = 2
    rows = (len(image_files) + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 7 * rows))
    if rows == 1:
        axes = [axes] if cols == 1 else axes
    else:
        axes = axes.flatten()
    
    for i, filename in enumerate(image_files):
        filepath = os.path.join(outputs_dir, filename)
        
        try:
            # Load and display image
            image = PILImage.open(filepath)
            
            # Extract info from filename (flux_model_timestamp.png)
            parts = filename.replace('.png', '').split('_')
            if len(parts) >= 3:
                model = parts[1]
                timestamp = parts[2]
                title = f"{model}\\n{filename}"
            else:
                title = filename
            
            ax = axes[i] if len(image_files) > 1 else axes
            # Convert PIL image to numpy array for matplotlib
            import numpy as np
            image_array = np.array(image)
            ax.imshow(image_array)
            ax.set_title(title, fontsize=12, fontweight='bold')
            ax.axis('off')
            
        except Exception as e:
            print(f"❌ Error loading {filename}: {e}")
    
    # Hide unused subplots
    for i in range(len(image_files), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Show file details
    print("\\nImage Details:")
    for filename in image_files[:5]:  # Show details for first 5 images
        filepath = os.path.join(outputs_dir, filename)
        stat = os.stat(filepath)
        size_mb = stat.st_size / (1024 * 1024)
        mod_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(stat.st_mtime))
        print(f"  📄 {filename} - {size_mb:.1f}MB - {mod_time}")
    
    if len(image_files) > 5:
        print(f"  ... and {len(image_files) - 5} more images")

# Image gallery button
gallery_button = widgets.Button(
    description='🖼️ View Image Gallery',
    button_style='info',
    icon='images',
    layout=widgets.Layout(width='200px', height='40px')
)

gallery_button.on_click(lambda b: display_image_gallery())
display(gallery_button)

### Step 4: FLUX Kontext Image Editing Studio

#### Background: Understanding Instruction-Based Image Editing

**What is FLUX Kontext?**

FLUX Kontext represents a paradigm shift in image editing technology:

- **Instruction-Based**: Uses natural language commands instead of manual selection tools
- **Surgical Precision**: Modifies only specified elements while preserving everything else
- **AI-Powered**: Understands context, objects, lighting, and artistic styles
- **Non-Destructive**: Original image remains unchanged; creates new edited versions

**Traditional vs Kontext Editing:**

| Traditional Photo Editing | FLUX Kontext |
|---------------------------|--------------|
| Manual selection tools (lasso, brush) | Natural language instructions |
| Complex layer management | Single instruction execution |
| Requires technical expertise | Accessible to anyone |
| Time-intensive (hours) | Fast execution (30-60 seconds) |
| Limited by tool capabilities | Limited by imagination |
| Static results | Multiple interpretation possibilities |

**How Kontext Works:**

1. **Input Analysis**: AI analyzes the source image to understand composition, objects, lighting
2. **Instruction Parsing**: Natural language processing interprets editing commands
3. **Selective Modification**: AI identifies specific areas that need changes
4. **Context Preservation**: Maintains lighting, perspective, and style consistency
5. **Result Generation**: Creates edited image while preserving unspecified elements

**Instruction Best Practices:**

**Good Instructions:**
- "Change the background to a sunset sky, keep the person unchanged"
- "Add professional studio lighting with soft shadows, preserve everything else"
- "Transform to watercolor painting style, maintain the subject and composition"

**Less Effective Instructions:**
- "Make it better" (too vague)
- "Change everything" (defeats the purpose of surgical editing)
- "Add some stuff" (unclear what to add)

**Key Principles:**

1. **Be Specific**: Clearly state what to change
2. **Preserve Context**: Explicitly mention what to keep unchanged
3. **Use Descriptive Language**: Include style, color, lighting details
4. **One Change at a Time**: Avoid complex multi-step instructions

**Business Applications:**

**E-commerce Photography:**
- Remove backgrounds for product catalogs
- Add lifestyle settings to product shots
- Enhance colors and lighting for better appeal
- Create multiple variants for A/B testing

**Professional Photography:**
- Fix lighting issues without reshooting
- Change backgrounds for different contexts
- Enhance colors and contrast
- Remove unwanted objects

**Marketing and Creative:**
- Adapt images for different campaigns
- Change seasonal elements (summer to winter)
- Apply brand-consistent styling
- Create variations for different platforms

**What the Kontext Code Does:**

- **Image Upload Handling**: Manages file uploads and base64 conversion
- **Template System**: Provides proven editing instructions
- **API Integration**: Sends images + instructions to Kontext NIM
- **Before/After Display**: Shows original and edited images side-by-side
- **Batch Processing**: Handles multiple editing operations
- **Metadata Tracking**: Saves editing history and parameters

**API Payload Format for Kontext:**

```json
{
  "prompt": "Your editing instruction",
  "image": "data:image/png;base64,YOUR_BASE64_STRING",
  "seed": 42,
  "steps": 50
}
```

**Key Differences from Regular FLUX:**

- **Image Parameter**: Uses `"image"` field with data URI format (`data:image/png;base64,...`)
- **No Mode Parameter**: Kontext doesn't use the `"mode"` field
- **Instruction Parameter**: Uses `prompt` field for editing instructions
- **Base64 Format**: Image must be provided as base64-encoded data URI string


In [ ]:
# FLUX Kontext NIM Deployment
# Deploy FLUX Kontext for instruction-based image editing

print("🚀 FLUX Kontext NIM Deployment")
print("Deploy FLUX Kontext for instruction-based image editing")
print("⏱️  Expected time: 3-5 minutes (may take up to 40 minutes due to Hugging Face rate limiting)")
print("📊 Resource requirements: 1 GPU per endpoint")
print()
print("🎯 DEPLOYING FLUX KONTEXT:")
print("  • Instruction-Based Image Editing: Edit images with natural language")
print("  • High Quality Results: Surgical precision modifications")
print("  • Non-Destructive: Preserves original image quality")
print()
print("⚠️  PREREQUISITE: Complete Lepton CLI authentication first!")
print()

# Kontext deployment configuration
kontext_endpoint_name_widget = widgets.Text(
    value='',
    placeholder='Leave empty for auto-generated name (flux-kontext)',
    description='Endpoint Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Reuse the same configuration widgets from FLUX.1-dev deployment
print("📋 SETUP REQUIREMENTS:")
print("1. 🔑 Hugging Face Token: REQUIRED for all FLUX NIMs (reuse from FLUX.1-dev setup)")
print("2. 🏢 Node Group: Your specific node group name")
print("3. 🔐 Image Pull Secrets: From Lepton Settings → Registries")
print("4. ⚠️  Accept FLUX.1-Kontext model license on Hugging Face:")
print("   https://huggingface.co/black-forest-labs/FLUX.1-Kontext-dev")
print()

display(kontext_endpoint_name_widget)

# Kontext deployment function
def deploy_kontext_nim():
    """Deploy FLUX Kontext NIM endpoint"""
    
    # Get configuration from the FLUX.1-dev deployment widgets (reuse them)
    node_group = node_group_input.value.strip() if 'node_group_input' in globals() else 'tme-nebius-h200-01'
    hf_token = hf_token_input.value.strip() if 'hf_token_input' in globals() else ''
    image_pull_secrets = image_pull_secrets_input.value.strip() if 'image_pull_secrets_input' in globals() else 'roclark-ngc'
    
    endpoint_name = kontext_endpoint_name_widget.value.strip()
    if not endpoint_name:
        endpoint_name = "flux-kontext"
    else:
        # Sanitize user-provided endpoint name
        endpoint_name = endpoint_name.replace('.', '-').replace('_', '-').lower()
    
    model = 'flux.1-kontext-dev'
    resource_type = 'gpu.1xh200'  # Use H200 for Kontext
    
    with kontext_deployment_output:
        kontext_deployment_output.clear_output(wait=True)
        
        print("🔍 FLUX Kontext Deployment")
        print("="*50)
        print("Deploying FLUX Kontext for instruction-based image editing")
        print()
        
        print(f"🎨 Deploying: {model}")
        print("-" * 40)
        
        # Check if CLI is available
        lepton_cmd = os.environ.get('LEPTON_CMD', '')
        if not lepton_cmd:
            print("❌ No working CLI command available")
            print("📋 Please complete the Lepton CLI setup first")
            return
        
        # Try deployment
        has_deployment_create = True  # Assume CLI is working
        
        if has_deployment_create:
            print("🚀 Attempting lep deployment create...")
            create_success, create_result = try_deployment_create(model, endpoint_name, resource_type, node_group, hf_token, image_pull_secrets)
            
            if create_success:
                print("✅ FLUX Kontext deployment successful!")
                print(f"📋 Result: {create_result}")
                print()
                print("🎉 FLUX Kontext is now ready for image editing!")
                print("📝 You can now use the image editing tools below.")
            else:
                print(f"❌ Deployment failed: {create_result}")
                print()
                print("💡 Common issues with FLUX Kontext:")
                print("  • HF Token required: Ensure you have a valid Hugging Face token")
                print("  • PERSISTENT RATE LIMITS: HF is aggressively rate limiting FLUX Kontext")
                print("    - Deployment may take 20-40 minutes with retry errors (this is normal)")
                print("    - The container will retry automatically until successful")
                print("    - Try deploying during off-peak hours for faster results")
                print("    - Consider using a different HF account/token if repeatedly failing")
                print("  • Resource limits: Check your Lepton GPU quota")
                print("  • Alternative: Use FLUX.1-dev for now (works reliably)")
                print()
                print("🔄 Manual deployment guide:")
                generate_corrected_deployment_guide(model, endpoint_name, resource_type, node_group, hf_token, image_pull_secrets)

# Kontext deployment button and output
kontext_deploy_button = widgets.Button(
    description='🚀 Deploy FLUX Kontext',
    button_style='warning',  # Orange color to distinguish from FLUX.1-dev
    icon='rocket',
    layout=widgets.Layout(width='200px', height='40px')
)

kontext_deployment_output = widgets.Output()

kontext_deploy_button.on_click(lambda b: deploy_kontext_nim())
display(kontext_deploy_button)
display(kontext_deployment_output)

print()
print("💡 After successful deployment:")
print("  • Deployment may take 3-40 minutes (HF rate limiting can cause delays)")
print("  • Once ready, update the endpoint URL in the image editing section below")
print("  • Start editing images with natural language instructions!")
print()
print("="*60)


In [ ]:

# FLUX Kontext editing infrastructure
def kontext_edit(image_base64, instruction, endpoint, ngc_api_key):
    """Generate edited image using FLUX Kontext with instruction-based editing"""
    
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }
    
    payload = {
        "prompt": instruction,
        "image": f"data:image/png;base64,{image_base64}",  # Correct format: 'image' parameter with data URI
        "seed": np.random.randint(0, 1000000),
        "steps": 50
    }
    
    start_time = time.time()
    
    try:
        nim_endpoint = endpoint.rstrip('/') + '/v1/infer'
        response = requests.post(nim_endpoint, headers=headers, json=payload, timeout=120)
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            generation_time = end_time - start_time
            
            # Parse Kontext response
            if 'artifacts' in result and len(result['artifacts']) > 0:
                base64_image = result['artifacts'][0].get('base64')
                if base64_image:
                    return {
                        'success': True,
                        'image_base64': base64_image,
                        'generation_time': generation_time,
                        'instruction': instruction,
                        'seed': result['artifacts'][0].get('seed', payload['seed'])
                    }
            
            return {
                'success': False,
                'error': f"No image artifacts in Kontext response: {result}",
                'instruction': instruction
            }
        else:
            return {
                'success': False,
                'error': f"Kontext API Error: {response.status_code} - {response.text}",
                'instruction': instruction
            }
            
    except Exception as e:
        return {
            'success': False,
        }
def display_before_after(original_content, edited_base64, instruction):
    """Display before/after comparison of original and edited images"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))
    
    # Original image
    original_img = PILImage.open(io.BytesIO(original_content))
    ax1.imshow(original_img)
    ax1.set_title("Original", fontsize=14, fontweight='bold')
    ax1.axis('off')
    
    # Edited image
    try:
        edited_bytes = base64.b64decode(edited_base64)
        edited_img = PILImage.open(io.BytesIO(edited_bytes))
        ax2.imshow(edited_img)
        ax2.set_title(f"Edited: {instruction[:40]}{'...' if len(instruction) > 40 else ''}", 
                     fontsize=14, fontweight='bold')
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        return edited_img
    except Exception as e:
        ax2.text(0.5, 0.5, f"Error displaying edited image: {e}", 
                ha='center', va='center', transform=ax2.transAxes)
        ax2.axis('off')
        plt.tight_layout()
        plt.show()
        return None

print("✅ FLUX Kontext editing infrastructure ready!")

### Interactive Photo Editor Studio

Upload any photo and apply instant AI-powered edits with simple instructions. FLUX Kontext understands natural language commands and surgically modifies only what you specify.

In [ ]:
# Interactive Photo Editor Studio
print("📸 Interactive Photo Editor Studio")
print("=" * 50)

# FLUX Kontext Endpoint Configuration
print("🎨 FLUX Kontext Endpoint Configuration")
print("Configure your FLUX Kontext endpoint for image editing")
print()

# Kontext endpoint selection dropdown
kontext_endpoint_widget = widgets.Dropdown(
    options=[
        ('Custom Kontext Endpoint', 'custom')
    ],
    value='custom',
    description='Kontext Endpoint:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Custom Kontext endpoint input
custom_kontext_endpoint_widget = widgets.Text(
    value='',
    placeholder='Enter your FLUX Kontext endpoint URL (e.g., https://your-kontext-deployment.cloud.lepton.ai)',
    description='Custom Kontext URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Kontext NGC API key (can reuse the same one)
kontext_ngc_api_key_widget = widgets.Password(
    placeholder='Enter your NGC API key (same as regular FLUX)',
    description='NGC API Key:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

display(kontext_endpoint_widget)
display(custom_kontext_endpoint_widget)
display(kontext_ngc_api_key_widget)

print()
print("💡 Note: FLUX Kontext requires a separate endpoint from regular FLUX.1-dev")
print("🚀 Deploy FLUX Kontext using the deployment cell above if needed")
print()

# File upload widget
photo_upload = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='📸 Upload Photo',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Quick edit templates with proven instructions
edit_templates = {
    "🎨 Artistic Style": "Transform to watercolor painting style, keep the subject and composition unchanged",
    "🌅 Change Background": "Change the background to a beautiful sunset sky, keep the subject in exact same position",
    "💡 Studio Lighting": "Add professional studio lighting with soft shadows, keep everything else unchanged",
    "🖼️ Remove Background": "Remove the background completely, keep the subject unchanged",
    "✨ Enhance Colors": "Enhance colors and contrast to look more vibrant, keep composition unchanged",
    "🌟 Make Premium": "Make the image look more premium and professional, keep the subject unchanged"
}

# Create template buttons
template_buttons = []
for name, instruction in edit_templates.items():
    button = widgets.Button(
        description=name,
        button_style='info',
        layout=widgets.Layout(width='200px', margin='2px'),
        tooltip=instruction
    )
    template_buttons.append(button)

# Arrange buttons in grid
button_grid = widgets.GridBox(
    template_buttons,
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='repeat(3, 200px)',
        grid_gap='5px'
    )
)

# Custom instruction input
custom_instruction = widgets.Textarea(
    placeholder='Enter your custom editing instruction (e.g., "Add glasses to the person, keep everything else unchanged")',
    description='Custom Edit:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px', height='80px')
)

# Custom edit button
custom_button = widgets.Button(
    description='✨ Apply Custom Edit',
    button_style='warning',
    layout=widgets.Layout(width='200px')
)

# Display widgets
display(photo_upload)

# Debug function to test image upload
def test_image_upload():
    if photo_upload.value:
        print(f"✅ Image uploaded successfully!")
        print(f"   📁 Filename: {list(photo_upload.value.keys())[0]}")
        print(f"   📏 File size: {len(list(photo_upload.value.values())[0]['content'])} bytes")
        
        # Try to decode and display basic info
        try:
            import base64
            image_data = list(photo_upload.value.values())[0]['content']
            base64_data = base64.b64encode(image_data).decode('utf-8')
            print(f"   🔍 Base64 length: {len(base64_data)} characters")
            print(f"   ✅ Image ready for processing")
        except Exception as e:
            print(f"   ❌ Error processing image: {e}")
    else:
        print("❌ No image uploaded yet")
        print("💡 Please select an image file using the upload button above")

# Test button
test_upload_button = widgets.Button(
    description='🔍 Test Upload',
    button_style='info',
    layout=widgets.Layout(width='150px')
)
test_upload_button.on_click(lambda b: test_image_upload())
display(test_upload_button)

print("\n🎯 Quick Edit Templates:")
display(button_grid)
print("\n✏️ Custom Instructions:")
display(custom_instruction)
display(custom_button)

# Processing function
def process_photo_edit(instruction):
    """Process photo editing with given instruction"""
    
    # Check if photo is uploaded
    if not photo_upload.value:
        print("❌ Please upload a photo first!")
        return
    
    # Get current KONTEXT endpoint configuration (separate from regular FLUX)
    endpoint = kontext_endpoint_widget.value
    if endpoint == 'custom':
        endpoint = custom_kontext_endpoint_widget.value
    ngc_api_key = kontext_ngc_api_key_widget.value
    
    if not endpoint.strip():
        print("❌ Please configure NIM endpoint first!")
        return
    
    print(f"\\n🔄 Processing edit: {instruction[:60]}{'...' if len(instruction) > 60 else ''}")
    print("⏳ This may take 30-60 seconds...")
    
    try:
        # Get uploaded file content
        uploaded_file = photo_upload.value[0]
        file_content = uploaded_file['content']
        
        # Convert to base64
        image_base64 = image_to_base64(file_content)
        
        # Call FLUX Kontext
        result = kontext_edit(image_base64, instruction, endpoint, ngc_api_key)
        
        if result['success']:
            print(f"✅ Edit completed in {result['generation_time']:.2f}s")
            
            # Display before/after comparison
            edited_img = display_before_after(file_content, result['image_base64'], instruction)
            
            if edited_img:
                # Save edited image
                os.makedirs('examples/outputs', exist_ok=True)
                timestamp = int(time.time())
                filename = f"kontext_edit_{timestamp}.png"
                filepath = f"examples/outputs/{filename}"
                
                # Save image
                edited_bytes = base64.b64decode(result['image_base64'])
                with open(filepath, 'wb') as f:
                    f.write(edited_bytes)
                
                print(f"💾 Edited image saved: {filepath}")
                
                # Save edit metadata
                metadata = {
                    'instruction': instruction,
                    'generation_time': result['generation_time'],
                    'seed': result['seed'],
                    'timestamp': timestamp,
                    'original_filename': uploaded_file['name']
                }
                
                metadata_path = f"examples/outputs/kontext_edit_{timestamp}_metadata.json"
                with open(metadata_path, 'w') as f:
                    json.dump(metadata, f, indent=2)
                
                print("📝 Edit metadata saved for future reference")
        else:
            print(f"❌ Edit failed: {result['error']}")
            print("💡 Try:")
            print("  - Check NIM container is running: docker ps")
            print("  - Verify endpoint URL")
            print("  - Use more specific instructions")
            
    except Exception as e:
        print(f"❌ Error processing edit: {str(e)}")

# Bind template buttons to processing function
for i, (name, instruction) in enumerate(edit_templates.items()):
    def make_handler(inst):
        return lambda b: process_photo_edit(inst)
    template_buttons[i].on_click(make_handler(instruction))

# Bind custom instruction button
def on_custom_edit(button):
    instruction = custom_instruction.value.strip()
    if instruction:
        process_photo_edit(instruction)
    else:
        print("❌ Please enter a custom instruction first!")

custom_button.on_click(on_custom_edit)

print("\\n💡 Tips for better results:")
print("  • Be specific about what to change and what to keep")
print("  • Use phrases like 'keep the subject unchanged'")
print("  • Try multiple variations to see different interpretations")
print("  • FLUX Kontext works best with clear, actionable instructions")

### Summary & Next Steps: FLUX Kontext Editing

🎉 **You've explored advanced FLUX Kontext image editing capabilities!**

#### What You've Learned:
- ✅ **Instruction-based editing** with natural language commands
- ✅ **Professional photo enhancement** with surgical precision  
- ✅ **Product photography automation** for e-commerce applications
- ✅ **Before/after comparison workflows** for quality assessment
- ✅ **Batch processing techniques** for multiple variants

#### Key Differentiators:
- **🎯 Surgical Editing**: FLUX Kontext modifies only what you specify
- **🏢 Business Applications**: Real product photography enhancement
- **⚡ Speed**: Professional results in 30-60 seconds
- **🔄 Iterative Workflow**: Build on previous edits
- **💾 Production Ready**: Automatic saving and metadata tracking

#### FLUX Kontext vs Traditional Editing:
| Traditional Photo Editing | FLUX Kontext |
|---------------------------|--------------|
| Manual selection tools | Natural language instructions |
| Hours of work | 30-60 seconds |
| Requires expertise | Accessible to anyone |
| Limited creativity | AI-powered imagination |
| One result | Multiple interpretations |

#### Business Impact:
- **E-commerce**: Generate multiple product shots instantly
- **Marketing**: A/B test different visual styles
- **Content Creation**: Rapid prototyping of visual concepts
- **Photography**: Enhance existing shots without reshooting

#### Next Steps:
- 🎨 **Experiment** with your own photos and products
- 🔄 **Chain edits** for complex transformations  
- 📊 **A/B test** different styles for your use case
- 🚀 **Deploy** your own FLUX Kontext NIM for production use
- 📈 **Scale** with automated batch processing workflows

#### Resources for Further Learning:
- [NVIDIA NGC FLUX Kontext Container](https://catalog.ngc.nvidia.com/orgs/nim/teams/black-forest-labs/containers/flux.1-kontext-dev)
- [FLUX Kontext Documentation](https://docs.nvidia.com/nim/visual-genai/latest/api/flux.1-kontext-dev.html)
- [Black Forest Labs FLUX Models](https://huggingface.co/black-forest-labs)

**🚀 You now have hands-on experience with cutting-edge AI image editing technology that's transforming creative workflows worldwide!**

### Step 5: Deploy Your Own NIM Endpoint (Optional)

#### Background: Understanding NIM Deployment Architecture

**Why Deploy Your Own NIM?**
While local development is great for testing, production applications require scalable, cloud-based deployments:

- **Scalability**: Handle multiple concurrent requests
- **Reliability**: 99.9% uptime with redundancy and failover
- **Performance**: Dedicated GPU resources without local limitations
- **Security**: Isolated environments with proper authentication
- **Cost Efficiency**: Pay-per-use instead of maintaining local hardware

**Deployment Options:**

**1. Lepton DevPods** (Recommended):
- **Purpose**: Rapid cloud deployment of AI models
- **Benefits**: Pre-configured environments, automatic scaling, simple deployment
- **Use Case**: Production applications, API services, team collaboration
- **Cost Model**: Pay-per-GPU-hour, automatic resource management

**2. AWS/GCP/Azure**:
- **Purpose**: Enterprise-grade cloud deployments
- **Benefits**: Full control, enterprise compliance, existing cloud integration
- **Use Case**: Large-scale applications, enterprise environments
- **Cost Model**: Instance-based pricing, manual resource management

**3. On-Premises Kubernetes**:
- **Purpose**: Complete control and data sovereignty
- **Benefits**: Full customization, no data egress, compliance control
- **Use Case**: Regulated industries, high-security environments
- **Cost Model**: Hardware + maintenance costs

**Container Architecture:**
```
FLUX NIM Container
├── Base Image: NVIDIA NGC optimized runtime
├── Model Files: Pre-quantized FLUX weights
├── TensorRT Engines: GPU-optimized inference
├── API Server: REST API endpoint
├── Health Checks: Container monitoring
└── Configuration: Environment variables
```

**Deployment Workflow:**
1. **Container Selection**: Choose appropriate FLUX NIM variant
2. **Configuration Generation**: Create deployment manifests
3. **Registry Push**: Upload container to private registry (optional)
4. **Cloud Deployment**: Deploy to chosen cloud platform
5. **Health Verification**: Ensure API endpoints are responding
6. **Load Testing**: Validate performance under load
7. **Monitoring Setup**: Configure logging and metrics

**Environment Variables:**
- **`NGC_API_KEY`**: Required for container access
- **`NIM_PORT`**: API server port (default: 8000)
- **`WORKSPACE`**: Working directory for files
- **`GPU_MEMORY_FRACTION`**: GPU memory allocation

**Resource Requirements:**

| Model | GPU Memory | CPU Cores | System Memory | Storage |
|-------|------------|-----------|---------------|---------|
| FLUX.1-dev | 12GB VRAM | 4+ cores | 16GB RAM | 50GB |
| FLUX.1-kontext-dev | 15GB VRAM | 4+ cores | 16GB RAM | 60GB |

**Security Considerations:**
- **API Key Management**: Secure storage of NGC API keys
- **Network Security**: VPC/firewall configuration
- **Access Control**: Authentication and authorization
- **Data Privacy**: Image data handling and retention
- **Compliance**: GDPR, HIPAA, SOC2 requirements

**Cost Optimization:**
- **Auto-scaling**: Scale down during low usage
- **Spot Instances**: Use preemptible instances where possible
- **Regional Selection**: Choose optimal regions for latency/cost
- **Monitoring**: Track usage to optimize resource allocation

**What the Deployment Code Does:**
- **Configuration Generation**: Creates Lepton DevPod YAML manifests
- **Container Management**: Handles Docker builds and registry operations
- **Environment Setup**: Configures required environment variables
- **Deployment Scripts**: Provides ready-to-run deployment commands
- **Validation Tools**: Checks deployment health and functionality

**Generated Artifacts:**
- **`lepton_devpod.yaml`**: Kubernetes-style deployment manifest
- **`Dockerfile.nim`**: Custom container build instructions
- **`build_flux_nim.sh`**: Automated build script
- **Deployment documentation**: Step-by-step deployment guide

In [ ]:
# Deployment configuration widgets
deploy_registry_widget = widgets.Text(
    value='nvcr.io/your-username',
    placeholder='Enter your registry URL',
    description='Registry:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

nim_model_widget = widgets.Dropdown(
    options=[
        ('FLUX.1 Dev', 'flux.1-dev'),
        ('FLUX.1 Kontext Dev', 'flux.1-kontext-dev')
    ],
    value='flux.1-dev',
    description='Model:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

def generate_deployment_config():
    """Generate Lepton deployment configuration for FLUX NIM"""
    
    registry = deploy_registry_widget.value
    model = nim_model_widget.value
    
    # Generate Lepton DevPod config using verified container references
    lepton_config = {
        "apiVersion": "lepton.ai/v1",
        "kind": "DevPod",
        "metadata": {
            "name": f"flux-nim-{model}",
            "labels": {
                "app": "flux-nim",
                "model": model
            }
        },
        "spec": {
            "image": f"nvcr.io/nim/black-forest-labs/flux.1-dev:1.1.0" if model == 'flux.1-dev' else f"nvcr.io/nim/black-forest-labs/flux.1-kontext-dev:1.0.0",
            "resources": {
                "gpu": {
                    "type": "nvidia",
                    "count": 1
                },
                "cpu": {
                    "cores": 4
                },
                "memory": "16Gi"
            },
            "ports": [
                {
                    "name": "nim-api",
                    "port": 8000,
                    "protocol": "TCP"
                }
            ],
            "env": [
                {
                    "name": "NGC_API_KEY",
                    "value": "your-ngc-api-key"
                }
            ]
        }
    }
    
    # Save configuration
    import yaml
    
    os.makedirs('configs', exist_ok=True)
    
    with open('configs/lepton_devpod.yaml', 'w') as f:
        yaml.dump(lepton_config, f, default_flow_style=False)
    
    print(f"✅ Generated Lepton DevPod configuration for {model}")
    print(f"📁 Saved to: configs/lepton_devpod.yaml")
    print(f"\n🚀 Deploy with: lepton create --file configs/lepton_devpod.yaml")
    print(f"\n⚠️  Remember to update NGC_API_KEY in the config file!")
    print(f"\n💡 API endpoint will be: http://your-lepton-url/v1/infer")

print("🚀 Optional: Deploy Your Own FLUX NIM")
display(deploy_registry_widget)
display(nim_model_widget)

deploy_button = widgets.Button(
    description='Generate Deployment Config',
    button_style='warning',
    icon='rocket',
    layout=widgets.Layout(width='220px', height='40px')
)

deploy_button.on_click(lambda b: generate_deployment_config())
display(deploy_button)

### Summary & Next Steps

🎉 **You've explored NVIDIA NIMs with FLUX models!**

#### What You've Learned:
- ✅ How NIMs provide optimized model inference
- ✅ Performance differences between FLUX model variants
- ✅ How to integrate NIMs into your workflows
- ✅ Optional deployment patterns for your own endpoints

#### Key Takeaways:
- **FLUX Schnell**: Best for rapid prototyping and iteration
- **FLUX Dev**: Balanced quality/speed for most use cases
- **FLUX Kontext**: Highest quality for production outputs
- **NIMs**: Easier deployment than managing local models

#### Next Steps:
- 🎨 Experiment with different prompts and parameters
- 🔧 Deploy your own NIM endpoints for custom control
- 📈 Integrate NIMs into larger AI workflows
- 🚀 Explore other NIMs models beyond FLUX

#### Resources:
- [NVIDIA NGC Catalog](https://catalog.ngc.nvidia.com/)
- [FLUX Model Documentation](https://github.com/black-forest-labs/flux)
- [Lepton AI Documentation](https://docs.lepton.ai/)

Happy experimenting! 🚀